In [2]:
import cntk
import numpy
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import time

def create_reader(path, is_training, input_dim, num_label_classes):
    featureStream = cntk.io.StreamDef(field='features', shape=input_dim, is_sparse=False)
    deserailizer = cntk.io.CTFDeserializer(path, cntk.io.StreamDefs(features=featureStream))
    return cntk.io.MinibatchSource(deserailizer, randomize=is_training, max_sweeps=1, randomization_window_in_chunks=0)

def load_model(path):
    model = cntk.ops.functions.load_model(path)
    return model

def predict(model, data):
    return model.eval(data)

def do_predict(model_path, data_path, len_data):
    model = load_model(model_path)
    reader = create_reader(data_path, False, 3, 3)
    input_map = {
        input: reader.streams.features
    }
    all_predict_value = numpy.zeros((1,3), dtype=int)
    
    num_minibatches = len_data//100
    #print(len_data, num_minibatches)
    for i in range(len_data):
        data = reader.next_minibatch(1, input_map=input_map)
        feature_data = data[input].asarray()
        predict_value = predict(model, feature_data)
        all_predict_value = numpy.append(all_predict_value, predict_value, axis=0)
        
    print("Finish Prediction")
    return all_predict_value 

def img_to_txt(path):
    txt_name = "image_rgb_file.txt"
    img = Image.open(path)
    pixel_list = []
    for i in range(img.size[0]):    
        for j in range(img.size[1]):
            pixel_list.append(img.getpixel((i,j)))
            
    pixel_set = set(pixel_list)
    count = 1
    with open(txt_name, 'w') as outfile:
        for p in pixel_set:
            outfile.write("|no " + str(count) + "|features " + str(p[0]) + " " + str(p[1]) + " " + str(p[2]) + "\n")
            count+=1
    print("Finish Convert Image")
    
    return pixel_set, txt_name ,len(pixel_set)

def data_to_img(predict_value, pixel_set, image_path, img_name):
    i = 0
    combined_dict = {}
    for e in pixel_set:
        #print(e, predict_value[i])
        combined_dict[e] = predict_value[i]
        i+=1
    
    img = Image.open(image_path)
    pixels = img.load()
    
    for i in range(img.size[0]):    
        for j in range(img.size[1]):
            key = img.getpixel((i,j))
            #print(img.getpixel((i,j)), "=>" ,combined_dict[key])
            R = int(combined_dict[key][0])
            G = int(combined_dict[key][1])
            B = int(combined_dict[key][2])
            
            R = correct_range(R)
            G = correct_range(G)
            B = correct_range(B)
            
            pixels[i,j] = (R, G, B)
            
    img.save(img_name)
    print("Finish Write Image")
    
def correct_range(pixel_value):
    if pixel_value < 0:
        pixel_value = 0
        
    if pixel_value > 255:
        pixel_value = 255
        
    return pixel_value
    
def create_img(model_path, image_path, output_image_name):
    pixel_set, pixel_txt_path, len_data = img_to_txt(image_path)
    predict_value = do_predict(model_path, pixel_txt_path, len_data)
    predict_value = numpy.delete(predict_value, 0, 0)
    predict_value = predict_value.round()
    data_to_img(predict_value, pixel_set, image_path, output_image_name)
    

images_name = ['Arm(S8+)_resize', 'ATM(S8+)_resize', 'Bucket(S8+)_resize', 'Snack(S8+)_resize', 'TheTwo(S8+)_resize']
experiment_no = "25"
model_no = "2"

root_image_path = "./example_image/"
model_path = "./saved_model/saved_model_exp_" + experiment_no + "/model-set-" + model_no + ".model"

for name in images_name:
    image_path = root_image_path + name + ".jpg"
    output_image_name = "./example_image/iPad_s8_augmented_img/" + name + "_Edited_By_exp-" + experiment_no + "_model-" + model_no + ".jpg"
    start_time = time.time()
    create_img(model_path, image_path, output_image_name)
    print("Finish : ", name)
    print("--- %s seconds ---" % (time.time() - start_time))

Finish Convert Image
Finish Prediction
Finish Write Image
Finish :  Arm(S8+)_resize
--- 57.42543411254883 seconds ---
Finish Convert Image
Finish Prediction
Finish Write Image
Finish :  ATM(S8+)_resize
--- 268.88801860809326 seconds ---
Finish Convert Image
Finish Prediction
Finish Write Image
Finish :  Bucket(S8+)_resize
--- 192.10580921173096 seconds ---
Finish Convert Image
Finish Prediction
Finish Write Image
Finish :  Snack(S8+)_resize
--- 199.2173011302948 seconds ---
Finish Convert Image
Finish Prediction
Finish Write Image
Finish :  TheTwo(S8+)_resize
--- 67.26924681663513 seconds ---
